In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import utils
from tensorflow.keras import mixed_precision
import os
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
mixed_precision.set_global_policy('mixed_float16')
import logging
tf.get_logger().setLevel(logging.ERROR)

2024-07-25 17:53:27.282976: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-25 17:53:27.308454: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-25 17:53:27.308479: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-25 17:53:27.308497: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-25 17:53:27.313866: I tensorflow/core/platform/cpu_feature_g

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 4070 Laptop GPU, compute capability 8.9


2024-07-25 17:53:28.831147: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-25 17:53:28.856217: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-25 17:53:28.858857: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [2]:
"""
#Batching using prefetch
train_data_casted = train_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE).shuffle(buffer_size = 1000).batch(batch_size = 32).prefetch(buffer_size = tf.data.AUTOTUNE)
test_data_casted = test_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE).batch(32).prefetch(tf.data.AUTOTUNE)
"""

'\n#Batching using prefetch\ntrain_data_casted = train_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE).shuffle(buffer_size = 1000).batch(batch_size = 32).prefetch(buffer_size = tf.data.AUTOTUNE)\ntest_data_casted = test_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE).batch(32).prefetch(tf.data.AUTOTUNE)\n'

In [3]:
fundus_train = "/home/thefilthysalad/PycharmProjects/eye_detection_fundus_dataset/Dataset/split/train"
fundus_test = "/home/thefilthysalad/PycharmProjects/eye_detection_fundus_dataset/Dataset/split/test"


BATCH_SIZE = 32
IMG_SIZE = (224, 224)


In [4]:
print(os.listdir(fundus_train))

['glaucoma', 'mild_dr', 'hypertensive_retinopathy', 'normal_fundus', 'cataract', 'moderate_dr', 'proliferate_dr', 'severe_dr', 'dry_amd', 'wet_amd', 'pathological_myopia']


In [5]:
IMG_HEIGHT, IMG_WIDTH = 224, 224

In [6]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    fundus_train,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=True,
    seed=123,
    validation_split=0.3,
    subset='training'
)

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    fundus_train,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=False,
    seed=123,
    validation_split=0.3,
    subset='validation'
)
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    fundus_test,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=False,
    seed=123,

)

Found 16269 files belonging to 11 classes.
Using 11389 files for training.


2024-07-25 17:53:32.449723: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-25 17:53:32.451303: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-25 17:53:32.452603: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Found 16269 files belonging to 11 classes.
Using 4880 files for validation.
Found 5428 files belonging to 11 classes.


In [7]:
train_dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 11), dtype=tf.float32, name=None))>

In [8]:
#Using tf prefetch dataset
preprocess_input = tf.keras.applications.inception_v3.preprocess_input

In [9]:
train_datagen = train_dataset.map(lambda x, y: (preprocess_input(x), y))
val_datagen = validation_dataset.map(lambda x, y: (preprocess_input(x), y))
test_datagen = test_dataset.map(lambda x, y: (preprocess_input(x), y))

In [10]:
train_datagen

<_MapDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 11), dtype=tf.float32, name=None))>

In [11]:
import tensorflow.keras.applications as apps
IMG_WIDTH, IMG_HEIGHT = 224, 224
base_model = apps.InceptionV3(weights = 'imagenet', include_top = False, input_shape = (IMG_WIDTH, IMG_HEIGHT, 3))
base_model.trainable = False

In [12]:

for i in base_model.layers:
    print(f'Layer: {i.name}, {i.trainable}')

Layer: input_1, False
Layer: conv2d, False
Layer: batch_normalization, False
Layer: activation, False
Layer: conv2d_1, False
Layer: batch_normalization_1, False
Layer: activation_1, False
Layer: conv2d_2, False
Layer: batch_normalization_2, False
Layer: activation_2, False
Layer: max_pooling2d, False
Layer: conv2d_3, False
Layer: batch_normalization_3, False
Layer: activation_3, False
Layer: conv2d_4, False
Layer: batch_normalization_4, False
Layer: activation_4, False
Layer: max_pooling2d_1, False
Layer: conv2d_8, False
Layer: batch_normalization_8, False
Layer: activation_8, False
Layer: conv2d_6, False
Layer: conv2d_9, False
Layer: batch_normalization_6, False
Layer: batch_normalization_9, False
Layer: activation_6, False
Layer: activation_9, False
Layer: average_pooling2d, False
Layer: conv2d_5, False
Layer: conv2d_7, False
Layer: conv2d_10, False
Layer: conv2d_11, False
Layer: batch_normalization_5, False
Layer: batch_normalization_7, False
Layer: batch_normalization_10, False
Lay

In [13]:
len(base_model.layers)

311

In [14]:
No_of_classes = len(os.listdir(fundus_train))
No_of_classes

11

In [15]:
from tensorflow.keras.layers import Dense, GlobalAvgPool2D, Input, Dropout
from tensorflow.keras.models import Model
inputs = Input(shape = (IMG_HEIGHT, IMG_WIDTH, 3), name = 'Input_layer')
x = base_model(inputs, training = False)
x = layers.GlobalAvgPool2D()(x)

x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
Outputs = Dense(No_of_classes, activation = 'softmax', dtype = tf.float32)(x)

model_1 = Model(inputs, Outputs, name = 'EfficientB7')

model_1.summary()

Model: "EfficientB7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 512)               262656    
                                                       

In [20]:
#Model checkpointing
from tensorflow.keras.callbacks import ModelCheckpoint
model_1chkpt = ModelCheckpoint(filepath = os.path.join('Trained_Models','InceptionV3'), save_weights_only = False, save_best_only = True, verbose = 1)

In [21]:
from keras.src.callbacks import ReduceLROnPlateau

lr_scheduler = ReduceLROnPlateau(factor=0.3, patience=2) 

In [22]:
#model compilation
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.001), metrics = ['accuracy'])

In [ ]:
history_1 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = 10, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 1/10
356/356 [==============================] - ETA: 0s - loss: 1.4852 - accuracy: 0.4501
Epoch 1: val_loss improved from inf to 1.44878, saving model to Trained_Models/InceptionV3
356/356 [==============================] - 27s 69ms/step - loss: 1.4852 - accuracy: 0.4501 - val_loss: 1.4488 - val_accuracy: 0.4059 - lr: 0.0010
Epoch 2/10
355/356 [============================>.] - ETA: 0s - loss: 1.3989 - accuracy: 0.4740
Epoch 2: val_loss improved from 1.44878 to 1.29696, saving model to Trained_Models/InceptionV3
356/356 [==============================] - 24s 67ms/step - loss: 1.3988 - accuracy: 0.4740 - val_loss: 1.2970 - val_accuracy: 0.4877 - lr: 0.0010
Epoch 3/10
355/356 [============================>.] - ETA: 0s - loss: 1.3461 - accuracy: 0.4823
Epoch 3: val_loss did not improve from 1.29696
356/356 [==============================] - 14s 38ms/step - loss: 1.3456 - accuracy: 0.4824 - val_loss: 1.3721 - val_accuracy: 0.4299 - lr: 0.0010
Epoch 4/10
354/356 [=====================

In [20]:
base_model.trainable = True

In [21]:
for layer in base_model.layers[:-50]:
    layer.trainable = False

In [22]:
for layer in base_model.layers:
    print(layer.trainable)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

In [23]:
start_epoch = 5
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.001), metrics = ['accuracy'])
model_1.summary()

Model: "EfficientB7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 MobilenetV3large (Function  (None, 7, 7, 960)         2996352   
 al)                                                             
                                                                 
 global_average_pooling2d (  (None, 960)               0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 256)               246016    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               

In [24]:
history_50 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+10, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 6/15
355/356 [============================>.] - ETA: 0s - loss: 0.9169 - accuracy: 0.6364
Epoch 6: val_loss improved from 0.70975 to 0.69144, saving model to Trained_Models/MobilenetV3L
356/356 [==============================] - 20s 48ms/step - loss: 0.9165 - accuracy: 0.6362 - val_loss: 0.6914 - val_accuracy: 0.6672 - lr: 0.0010
Epoch 7/15
355/356 [============================>.] - ETA: 0s - loss: 0.7614 - accuracy: 0.6794
Epoch 7: val_loss improved from 0.69144 to 0.51389, saving model to Trained_Models/MobilenetV3L
356/356 [==============================] - 17s 47ms/step - loss: 0.7610 - accuracy: 0.6797 - val_loss: 0.5139 - val_accuracy: 0.6303 - lr: 0.0010
Epoch 8/15
355/356 [============================>.] - ETA: 0s - loss: 0.6618 - accuracy: 0.7116
Epoch 8: val_loss improved from 0.51389 to 0.48093, saving model to Trained_Models/MobilenetV3L
356/356 [==============================] - 18s 50ms/step - loss: 0.6617 - accuracy: 0.7117 - val_loss: 0.4809 - val_accuracy: 0.7514

In [25]:
start_epoch = 15
for layer in base_model.layers[-100:]:
    layer.trainable = True

In [26]:
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.0003), metrics = ['accuracy'])
model_1.summary()

Model: "EfficientB7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 MobilenetV3large (Function  (None, 7, 7, 960)         2996352   
 al)                                                             
                                                                 
 global_average_pooling2d (  (None, 960)               0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 256)               246016    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               

In [27]:
history_100 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+15, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 16/30
356/356 [==============================] - ETA: 0s - loss: 0.5453 - accuracy: 0.7916
Epoch 16: val_loss did not improve from 0.27371
356/356 [==============================] - 16s 32ms/step - loss: 0.5453 - accuracy: 0.7916 - val_loss: 0.3833 - val_accuracy: 0.8607 - lr: 3.0000e-04
Epoch 17/30
355/356 [============================>.] - ETA: 0s - loss: 0.4776 - accuracy: 0.8000
Epoch 17: val_loss did not improve from 0.27371
356/356 [==============================] - 13s 37ms/step - loss: 0.4771 - accuracy: 0.8003 - val_loss: 0.3473 - val_accuracy: 0.8738 - lr: 3.0000e-04
Epoch 18/30
356/356 [==============================] - ETA: 0s - loss: 0.6035 - accuracy: 0.7815
Epoch 18: val_loss did not improve from 0.27371
356/356 [==============================] - 13s 37ms/step - loss: 0.6035 - accuracy: 0.7815 - val_loss: 0.3543 - val_accuracy: 0.8594 - lr: 3.0000e-04
Epoch 19/30
355/356 [============================>.] - ETA: 0s - loss: 0.4299 - accuracy: 0.8150
Epoch 19: val_loss

In [28]:
start_epoch = 30
for layer in base_model.layers[-200:]:
    layer.trainable = True

In [29]:
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.00009), metrics = ['accuracy'])
model_1.summary()

Model: "EfficientB7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 MobilenetV3large (Function  (None, 7, 7, 960)         2996352   
 al)                                                             
                                                                 
 global_average_pooling2d (  (None, 960)               0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 256)               246016    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               

In [30]:
history_200 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+15, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 31/45
356/356 [==============================] - ETA: 0s - loss: 0.3254 - accuracy: 0.8564
Epoch 31: val_loss did not improve from 0.15543
356/356 [==============================] - 27s 48ms/step - loss: 0.3254 - accuracy: 0.8564 - val_loss: 0.1932 - val_accuracy: 0.9262 - lr: 9.0000e-05
Epoch 32/45
355/356 [============================>.] - ETA: 0s - loss: 0.3095 - accuracy: 0.8568
Epoch 32: val_loss did not improve from 0.15543
356/356 [==============================] - 14s 39ms/step - loss: 0.3094 - accuracy: 0.8570 - val_loss: 0.2302 - val_accuracy: 0.9232 - lr: 9.0000e-05
Epoch 33/45
355/356 [============================>.] - ETA: 0s - loss: 0.3425 - accuracy: 0.8511
Epoch 33: val_loss did not improve from 0.15543
356/356 [==============================] - 14s 40ms/step - loss: 0.3423 - accuracy: 0.8513 - val_loss: 0.2257 - val_accuracy: 0.9205 - lr: 9.0000e-05
Epoch 34/45
355/356 [============================>.] - ETA: 0s - loss: 0.2941 - accuracy: 0.8718
Epoch 34: val_loss

In [31]:
start_epoch = 45
for layer in base_model.layers[-250:]:
    layer.trainable = True

In [32]:
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.0000015), metrics = ['accuracy'])
model_1.summary()

Model: "EfficientB7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 MobilenetV3large (Function  (None, 7, 7, 960)         2996352   
 al)                                                             
                                                                 
 global_average_pooling2d (  (None, 960)               0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 256)               246016    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               

In [ ]:
history_250 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+15, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler])

Epoch 46/60


In [7]:
MobileNet_Best = tf.keras.models.load_model("/home/thefilthysalad/PycharmProjects/eye_detection_fundus_dataset/ML_Models/Trained_Models/MobilenetV3L")


In [11]:
MobileNet_Best.evaluate(test_datagen)

2024-07-25 17:52:20.965819: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8600
2024-07-25 17:52:21.202099: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


170/170 [==============================] - 5s 18ms/step - loss: 0.4258 - accuracy: 0.8493


[0.42583829164505005, 0.8492999076843262]